In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LeNet5_LERU_ENH
import tf_general as tfg
import numpy as np
import aws_boto3

FLAGS = tf.flags.FLAGS

try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 50000, 'epoch')
    tf.flags.DEFINE_integer('batch_size',200, 'batch size')
    tf.flags.DEFINE_integer('test_size', 200, 'test size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_float('keep_prob', 0.5, 'keep prob for drop lay')
    tf.flags.DEFINE_boolean('augument', True, 'if image augument is applied')
                            
    tf.flags.DEFINE_float('ckpt_frequency', 250, 'frequency to save checkpoint')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

print("learn Rate =",FLAGS.lr, "keep Prob=", FLAGS.keep_prob, " image augument=",FLAGS.augument)   
CONTINUE = 0

parameters were defined.
learn Rate = 0.01 keep Prob= 0.5  image augument= True


## Generate Checkpoint dir and Log dir

- Checkpoint dir is saved in variable **ckpt_dir**, if the dir doesn't exist then create it. 
- Log file dir is saved in variable **ckpt_dir**, if the dir doesn't exist then create it. 

In [2]:
ckpt_dir = 'ckpt_7/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
log_dir = 'log_7/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

## Input layer 
Use the data feeder to provide the training data. Therefore we define a placeholder with the same structure of the input data. 
The CIFAR-10 data is 60000 RGB images with each size is 32x32. The data structure should be[batchsize, 32,32,3]. Input channels are 3.  

In [3]:
with tf.device('/cpu:0'):
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None,32,32,3], name='x_input')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        y_ = tf.placeholder(tf.int64, [None], name='labels')

In [4]:
with tf.device('/cpu:1'):
    with tf.name_scope('prediction'):
        le_net5 = LeNet5_LERU_ENH(x, keep_prob)
        y = le_net5.prediction
        """training parameters log
        x_norm = le_net5.x_norm
        conv1 = le_net5.conv1
        pool1 = le_net5.pool1
        conv2 = le_net5.conv2
        pool2 = le_net5.pool2
        fc1 = le_net5.fc1
        fc2 = le_net5.fc2
        fc3 = le_net5.fc3
        """

conv_1:  (?, 28, 28, 18)
pool_1:  (?, 14, 14, 18)
conv_2:  (?, 10, 10, 48)
pool_2:  (?, 5, 5, 48)
flat_1: (?, 1200)
fc_1:  (?, 120)
fc_2:  (?, 84)
fc_3:  (?, 10)
drop_out:  (?, 10)
prediction:  (?, 10)


In [5]:
with tf.device('/cpu:2'):
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                  labels=y_, 
                                                                                  name="cross_entropy_per_example"))

In [6]:
with tf.device('/cpu:3'):
    with tf.name_scope('train_step'):
        train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [7]:
with tf.device('/cpu:0'):
    prediction =tf.argmax(y, 1)
    accuracy = tf.reduce_mean(tf.cast( tf.equal(prediction,y_), tf.float32))

In [ ]:
def SaveCheckpoint2S3(ori_dir,des_dir):
    bucketname = 'sagemaker-deechean-dl' 
    for file in os.listdir(file_dir):
        if os.path.isfile(file_dir+file):
            print('save files to s3')
            aws_boto3.upload_file(file_dir+file, bucketname,des_dir+file)

In [ ]:
import time

if __name__ == '__main__':
    data = cifar10();
    
    with tf.Session(config=tf.ConfigProto(
                            device_count={"CPU":4},
                            inter_op_parallelism_threads=1,
                            intra_op_parallelism_threads=1)) as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep=1)
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint(ckpt_dir)
            saver.restore(sess,model_file)
        for i in range(CONTINUE, FLAGS.epoch):
            train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size,FLAGS.augument)
            loss, _,accuracy_rate = sess.run([cross_entropy, train_step, accuracy], 
                                             feed_dict={keep_prob: FLAGS.keep_prob, x:train_image, y_:train_label})
            
            tfg.savelog(log_dir,'train_accuracy',i, accuracy_rate)  
            tfg.savelog(log_dir,'train_loss',i, loss)   
            if (i+1) % FLAGS.ckpt_frequency == 0:  #保存预测模型
                saver.save(sess,ckpt_dir+'cifar10_'+str(i+1)+'.ckpt',global_step=i+1)
                #SaveCheckpoint2S3()
                acc_accuracy = 0
                for j in range(int(10000/FLAGS.test_size)):                    
                    test_image, test_label,test_index = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate, output = sess.run([accuracy,prediction],
                                                     feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                    acc_accuracy += accuracy_rate
                    tfg.savelog(log_dir,'test_index',i, test_index)
                    #tfg.savelog(log_dir,'x_norm',i, x_norm_value)  
                    #tfg.savelog(log_dir,'conv1',i, conv1_value)
                    #tfg.savelog(log_dir,'pool1',i, pool1_value)                     
                    #tfg.savelog(log_dir,'conv2',i, conv2_value)   
                    #tfg.savelog(log_dir,'pool2',i, pool2_value)   
                    #tfg.savelog(log_dir,'fc1',i, fc1_value) 
                    #tfg.savelog(log_dir,'fc2',i, fc2_value) 
                    #tfg.savelog(log_dir,'fc3',i, fc3_value)                       
                    tfg.savelog(log_dir,'output',i, output)
                accuracy_rate = acc_accuracy/10000*FLAGS.test_size
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + 
                      ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%')
                tfg.savelog(log_dir,'test_accuracy',i, accuracy_rate)            
    tf.reset_default_graph()

train set length: 50000
test set length: 10000
2019-06-09 14:01:10 iter 249, Test accuracy:13.78%
2019-06-09 14:08:33 iter 499, Test accuracy:13.62%
2019-06-09 14:15:26 iter 749, Test accuracy:16.98%
2019-06-09 14:22:13 iter 999, Test accuracy:17.98%
2019-06-09 14:30:37 iter 1249, Test accuracy:19.4%
2019-06-09 14:36:56 iter 1499, Test accuracy:20.11%
2019-06-09 14:50:11 iter 1749, Test accuracy:21.93%
2019-06-09 15:06:25 iter 1999, Test accuracy:22.16%
2019-06-09 15:25:14 iter 2249, Test accuracy:23.2%
2019-06-09 15:41:37 iter 2499, Test accuracy:23.04%
2019-06-09 15:58:19 iter 2749, Test accuracy:24.14%
2019-06-09 16:15:37 iter 2999, Test accuracy:24.21%
2019-06-09 16:32:02 iter 3249, Test accuracy:24.5%
2019-06-09 16:48:58 iter 3499, Test accuracy:25.57%
2019-06-09 17:04:23 iter 3749, Test accuracy:25.86%
2019-06-09 17:20:13 iter 3999, Test accuracy:26.88%
2019-06-09 17:35:53 iter 4249, Test accuracy:27.58%
2019-06-09 17:53:55 iter 4499, Test accuracy:27.69%
2019-06-09 18:10:19 iter